In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import csv
plt.style.use('fivethirtyeight')

In [3]:
#company="MCD/MCD"
company="AAPL/AAPL"
#company="KO/KO"

#company_name="MCD"
company_name="AAPL"
#company_name="KO"

non_place_data = pd.read_csv(company+"_scraped_tweets.csv", header=0,            
                  names=["date","tweet","tweet_id"])

non_place_data['date'] = non_place_data['date'].replace({'date':''}) #remove redundant date
non_place_data['tweet'] = non_place_data['tweet'].replace({'tweet':''}) #remove redundant tweet
non_place_data['tweet_id'] = non_place_data['tweet_id'].replace({'tweet_id':''}) #remove redundant tweet_id
non_place_data['date'].replace('', np.nan, inplace=True) #change null data to np.nan
non_place_data.dropna(axis=0, how='any',inplace=True) #remove all null data

non_place_data.to_csv(company+"_placed_data.csv",index=False) #output the csv file by cleaning data in first step
abc=pd.read_csv(company+"_placed_data.csv")

non_place_data.sort_values('date',ascending=False,inplace=True)



                           date  \
75.0  2022-03-04 22:15:45+00:00   
82.0  2022-03-04 21:36:50+00:00   
50.0  2022-03-04 21:30:00+00:00   
54.0  2022-03-04 20:49:50+00:00   
61.0  2022-03-04 20:10:18+00:00   
...                         ...   
49.0  2021-03-05 02:09:08+00:00   
28.0  2021-03-05 01:07:58+00:00   
23.0  2021-03-05 00:55:29+00:00   
15.0  2021-03-05 00:46:36+00:00   
26.0  2021-03-05 00:04:51+00:00   

                                                  tweet  \
75.0  Apple Inc price at close, 2022-03-04, is 163.1...   
82.0  $AAPL  8-K  Apple   (4 Mar, 4:35pm)\n\n📰 Depar...   
50.0  $AAPL closed today at $163.17.\n\nIf you bough...   
54.0  @pizdets17 We can just shave that off of #AAPL...   
61.0  @Penny_Lane_BBM Great!next  few weeks should b...   
...                                                 ...   
49.0  Feels right now #stockmarketcrash  #tsla #aapl...   
28.0  Lotto Friday is about to be fun tomorrow! \n\n...   
23.0  Mirando el semanal me quedo con  #byma ant

In [4]:
header = ['Tweet_Id','Date','Tweet']
with open(company+"_cleaned.csv",'w',encoding='utf-8', newline='') as data_csv:
    write=csv.writer(data_csv)
    write.writerow(header)
    
    for index,row in non_place_data.iterrows():
        stre=row["tweet"]
        if(type(stre)==str):
            clear_string = re.sub('[^a-zA-Z0-9]',' ',stre)
        else:
            clear_string=""
        import_array=[row["tweet_id"],row["date"],clear_string]
        write.writerow(import_array)


        
Test3 = pd.read_csv(company+"_cleaned.csv")
Test3['Date']= Test3['Date'].str.split(" ", expand = True)[0] #reformat the data
Test3['Date'] = Test3['Date'].str.replace('/','-')

adata=pd.DataFrame(columns=['Date','Tweet'])
total=100
index=0
for index,row in Test3.iterrows():
    stre=row["Tweet"]
    my_new_string = re.sub('[^a-zA-Z0-9]', ' ', stre)
    temp_df = pd.DataFrame([[Test3["Date"].iloc[index], 
                            my_new_string]], columns = ['Date','Tweet'])
    adata = pd.concat([adata, temp_df], axis = 0).reset_index(drop = True)
    # index=index+1
#print(cdata.dtypes)


In [6]:
adata["Comp"] = ''
adata["Negative"] = ''
adata["Neutral"] = ''
adata["Positive"] = ''

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sentiment_i_a = SentimentIntensityAnalyzer()
for indexx, row in adata.T.iteritems():
    try:
        sentence_i = unicodedata.normalize('NFKD', adata.loc[indexx, 'Tweet'])
        sentence_sentiment = sentiment_i_a.polarity_scores(sentence_i)
        adata['Comp'].loc[indexx] = sentence_sentiment['compound']
        adata['Negative'].loc[indexx] = sentence_sentiment['neg']
        adata['Neutral'].loc[indexx] = sentence_sentiment['neu']
        adata['Positive'].loc[indexx] = sentence_sentiment['compound']
    except TypeError:
        print (adata.loc[indexx, 'Tweet'])
        print (indexx)

             Date                                              Tweet    Comp  \
0      2022-03-04  Apple Inc price at close  2022 03 04  is 163 1...     0.0   
1      2022-03-04   AAPL  8 K  Apple    4 Mar  4 35pm     Departu...  0.2732   
2      2022-03-04   AAPL closed today at  163 17   If you bought ... -0.0516   
3      2022-03-04   pizdets17 We can just shave that off of  AAPL...     0.0   
4      2022-03-04   Penny Lane BBM Great next  few weeks should b...  0.8126   
...           ...                                                ...     ...   
18713  2021-03-05  Feels right now  stockmarketcrash   tsla  aapl...     0.0   
18714  2021-03-05  Lotto Friday is about to be fun tomorrow     S...  0.5106   
18715  2021-03-05  Mirando el semanal me quedo con   byma antes q...     0.0   
18716  2021-03-05                         xom    aapl                    0.0   
18717  2021-03-05                        APompliano  AAPL  Microsoft     0.0   

      Negative Neutral Positive  
0    

In [7]:
aadata=pd.DataFrame(columns=['Date','Comp','Negative','Neutral','Positive'])
count=0
comp=0
neg=0
neu=0
pos=0
for i in range(0,len(adata)-1):
    get_date=adata.Date.iloc[i]
    next_date=adata.Date.iloc[i+1]
    if(str(get_date)==str(next_date)):
        comp=adata.Comp.iloc[i+1]+adata.Comp.iloc[i]
        neg=adata.Negative.iloc[i+1]+adata.Negative.iloc[i]
        neu=adata.Neutral.iloc[i+1]+adata.Neutral.iloc[i]
        pos=adata.Positive.iloc[i+1]+adata.Positive.iloc[i]
        count=count+1
    if(str(get_date)!=str(next_date)):
        comp=comp/count
        neg=neg/count
        neu=neu/count
        pos=pos/count
        temp_df = pd.DataFrame([[get_date, 
                                comp,neg,neu,pos]], columns = ['Date','Comp','Negative','Neutral','Positive'])
       
        aadata = pd.concat([aadata, temp_df], axis = 0).reset_index(drop = True) #error
        count=0
        comp=0
        neg=0
        neu=0
        pos=0
        

aadata

,Date,Comp,Negative,Neutral,Positive
0,2022-03-04,-0.006844,0.008125,0.049062,-0.006844
1,2022-03-03,0.011440,0.000000,0.045325,0.011440
2,2022-03-02,0.018612,0.000000,0.044463,0.018612
3,2022-03-01,0.042463,0.000000,0.048767,0.042463
4,2022-02-28,0.000000,0.000000,0.050000,0.000000
...,...,...,...,...,...
359,2021-03-10,0.007485,0.001415,0.032226,0.007485
360,2021-03-09,0.001005,0.001852,0.029279,0.001005
361,2021-03-08,0.002641,0.002319,0.022710,0.002641
362,2021-03-07,-0.010680,0.005133,0.057633,-0.010680


In [8]:
df_Company =pd.read_csv(company+"_Price.csv")
print("Show the price data")
print(df_Company)

aadata['Prices']=''

indx=0
for i in range (0,len(aadata)):
    for j in range (0,len(df_Company)):
        get_tweet_date=aadata.Date.iloc[i]
        get_stock_date=df_Company.Date.iloc[j]
        if(str(get_stock_date)==str(get_tweet_date)):
            #print(get_stock_date," ",get_tweet_date)
            # ccdata.set_value(i,'Prices',int(read_stock_p.Close[j]))
            aadata['Prices'].iloc[i] = df_Company.Close[j]
print(aadata)


#for i in range(0,len(new_dict)):
#    if(new_dict.Prices.iloc[i]==""):
#        new_dict.Prices.iloc[i]=new_dict.Prices.iloc[i-1]
#print(new_dict)

aadata['Prices'].replace('', np.nan, inplace=True) #change null data to np.nan
aadata.dropna(axis=0, how='any',inplace=True) #remove all null data
aadata.reset_index(drop=True, inplace=True)
print(aadata)

Show the price data
           Date        Open        High         Low       Close   Adj Close  \
0    2021-03-05  120.980003  121.940002  117.570000  121.419998  120.702599   
1    2021-03-08  120.930000  121.000000  116.209999  116.360001  115.672493   
2    2021-03-09  119.029999  122.059998  118.790001  121.089996  120.374542   
3    2021-03-10  121.690002  122.169998  119.449997  119.980003  119.271118   
4    2021-03-11  122.540001  123.209999  121.260002  121.959999  121.239410   
..          ...         ...         ...         ...         ...         ...   
248  2022-02-28  163.059998  165.419998  162.429993  165.119995  165.119995   
249  2022-03-01  164.699997  166.600006  161.970001  163.199997  163.199997   
250  2022-03-02  164.389999  167.360001  162.949997  166.559998  166.559998   
251  2022-03-03  168.470001  168.910004  165.550003  166.229996  166.229996   
252  2022-03-04  164.490005  165.550003  162.100006  163.169998  163.169998   

        Volume  
0    153766600

C:\Users\may\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


           Date      Comp  Negative   Neutral  Positive Prices
0    2022-03-04 -0.006844  0.008125  0.049062 -0.006844    163
1    2022-03-03  0.011440  0.000000  0.045325  0.011440    166
2    2022-03-02  0.018612  0.000000  0.044463  0.018612    166
3    2022-03-01  0.042463  0.000000  0.048767  0.042463    163
4    2022-02-28  0.000000  0.000000  0.050000  0.000000    165
..          ...       ...       ...       ...       ...    ...
359  2021-03-10  0.007485  0.001415  0.032226  0.007485    119
360  2021-03-09  0.001005  0.001852  0.029279  0.001005    121
361  2021-03-08  0.002641  0.002319  0.022710  0.002641    116
362  2021-03-07 -0.010680  0.005133  0.057633 -0.010680       
363  2021-03-06  0.000000  0.000000  0.117647  0.000000       

[364 rows x 6 columns]
           Date      Comp  Negative   Neutral  Positive  Prices
0    2022-03-04 -0.006844  0.008125  0.049062 -0.006844   163.0
1    2022-03-03  0.011440  0.000000  0.045325  0.011440   166.0
2    2022-03-02  0.018612  0

In [9]:
aaadata=pd.DataFrame(columns=['Date','trend','Comp','Negative','Neutral','Positive','Prices'])

get_date=aadata.Date.iloc[0]
comp2=aadata.Comp.iloc[0]
neg2=aadata.Negative.iloc[0]
neu2=aadata.Neutral.iloc[0]
pos2=aadata.Positive.iloc[0]
prices2=aadata.Prices.iloc[0]

temp_df = pd.DataFrame([[get_date, 
                                '0',comp2,neg2,neu2,pos2,prices2]], columns = ['Date','trend','Comp','Negative','Neutral','Positive','Prices'])
aaadata = pd.concat([aaadata, temp_df], axis = 0).reset_index(drop = True)

In [11]:
aaadata.head()

,Date,trend,Comp,Negative,Neutral,Positive,Prices
0,2022-03-04,0,-0.006844,0.008125,0.049062,-0.006844,163.0


In [12]:
for i in range(1,len(aadata)):
    get_price=aadata.Prices.iloc[i]
    get_date=aadata.Date.iloc[i]
    last_price=aadata.Prices.iloc[i-1]
    last_date=aadata.Date.iloc[i-1]
    get_comp2=aadata.Comp.iloc[i]
    get_neg2=aadata.Negative.iloc[i]
    get_neu2=aadata.Neutral.iloc[i]
    get_pos2=aadata.Positive.iloc[i]
    if(last_price>get_price):
        temp_df = pd.DataFrame([[get_date, 
                                '0',get_comp2,get_neg2,get_neu2,get_pos2,get_price]], columns = ['Date','trend','Comp','Negative','Neutral','Positive','Prices'])
        aaadata = pd.concat([aaadata, temp_df], axis = 0).reset_index(drop = True)
    if(last_price<get_price):
        temp_df = pd.DataFrame([[get_date, 
                                '1',get_comp2,get_neg2,get_neu2,get_pos2,get_price]], columns = ['Date','trend','Comp','Negative','Neutral','Positive','Prices'])
        aaadata = pd.concat([aaadata, temp_df], axis = 0).reset_index(drop = True)
    if(last_price==get_price):
        temp_df = pd.DataFrame([[get_date, 
                                '2',get_comp2,get_neg2,get_neu2,get_pos2,get_price]], columns = ['Date','trend','Comp','Negative','Neutral','Positive','Prices'])
        aaadata = pd.concat([aaadata, temp_df], axis = 0).reset_index(drop = True) #error
        
print(aaadata)        

           Date trend      Comp  Negative   Neutral  Positive  Prices
0    2022-03-04     0 -0.006844  0.008125  0.049062 -0.006844   163.0
1    2022-03-03     1  0.011440  0.000000  0.045325  0.011440   166.0
2    2022-03-02     2  0.018612  0.000000  0.044463  0.018612   166.0
3    2022-03-01     0  0.042463  0.000000  0.048767  0.042463   163.0
4    2022-02-28     1  0.000000  0.000000  0.050000  0.000000   165.0
..          ...   ...       ...       ...       ...       ...     ...
247  2021-03-12     0  0.000000  0.000000  0.051282  0.000000   121.0
248  2021-03-11     2  0.000000  0.000000  0.040816  0.000000   121.0
249  2021-03-10     0  0.007485  0.001415  0.032226  0.007485   119.0
250  2021-03-09     1  0.001005  0.001852  0.029279  0.001005   121.0
251  2021-03-08     0  0.002641  0.002319  0.022710  0.002641   116.0

[252 rows x 7 columns]
